In [15]:
import numpy as np

a = np.random.random((10, 3))
print(a)
a[np.arange(10), np.argmin(np.absolute(a - 0.5), axis = 1)]

[[0.31401537 0.54776215 0.52162515]
 [0.17867841 0.64451046 0.47538683]
 [0.29451882 0.11954441 0.24911513]
 [0.08756961 0.40506172 0.67554488]
 [0.96127057 0.46957842 0.34442516]
 [0.05771486 0.21724326 0.71036336]
 [0.74037857 0.06635484 0.63445608]
 [0.88305861 0.195109   0.09362456]
 [0.79017661 0.03711512 0.2241542 ]
 [0.01126177 0.94432814 0.71637826]]


array([0.52162515, 0.47538683, 0.29451882, 0.40506172, 0.46957842,
       0.71036336, 0.63445608, 0.195109  , 0.2241542 , 0.71637826])

In [4]:
import numpy as np

a = np.random.random((6, 6))
print(a)
print(a.sum(axis = 1))
a.sum(axis = 1) / a[np.argmin(a, axis = 0), np.arange(6)]


[[0.43736282 0.69327443 0.9765034  0.9447012  0.41035665 0.5476671 ]
 [0.54318398 0.78042537 0.77265028 0.54342246 0.89179062 0.42122314]
 [0.39590372 0.00370061 0.38276484 0.12342959 0.06008085 0.36243898]
 [0.48466568 0.19188552 0.02229789 0.48462807 0.64798377 0.62851376]
 [0.24888463 0.79763211 0.47804734 0.82008022 0.83893831 0.40896162]
 [0.90453422 0.36202326 0.97954391 0.39702814 0.72358877 0.70842821]]
[4.00986559 3.95269584 1.32831859 2.45997469 3.59254423 4.07514653]


array([  16.11134277, 1068.12032568,   59.57148909,   19.93018597,
         59.7951631 ,   11.24367613])

In [123]:
import numpy as np 

x = np.array([666, 0, 12, 0, 10, 0, 0, 108, 7, 0, 177, 0])
print(x)
np.max(x[np.where(np.roll(x, 1) == 0)][1:])

[666   0  12   0  10   0   0 108   7   0 177   0]


177

In [173]:
import numpy as np

x = np.ones(10) 
print(x)
i = np.array([0, 1, 2, 3, 5, 5, 5, 8])
print(np.bincount(i))
np.pad(np.bincount(i), (0, len(x) - len(np.bincount(i))), 'constant') + x


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1 1 1 1 0 3 0 0 1]


array([2., 2., 2., 2., 1., 4., 1., 1., 2., 1.])

In [154]:
import numpy as np

n = 4
a = list(np.arange(n))
b = list(n + np.arange(n - 1))

dict(list((dict(zip(a, map(lambda x: list(x + (n - 1) * np.arange(x + 1)), a)))).items()) + list((dict(zip(b, map(lambda x: list((n-1) + ((x - n *(x // n)+1)) * n + (n-1) * np.arange(2*n - 1 - x)), b)))).items()))



{0: [0],
 1: [1, 4],
 2: [2, 5, 8],
 3: [3, 6, 9, 12],
 4: [7, 10, 13],
 5: [11, 14],
 6: [15]}

In [31]:
#в версии scipy 0.15.1, ссылка на документацию которой была в д/з, то ли как-то коряво было описано, что имеется в виду под 
#distortion, то ли в той версии scipy этот параметр вычислялся как-то совсем не так, как в последней. я для проверки своего
#результата импортировал kmeans из scipy, и, видимо, импортировалась более новая версия библиотеки. 
#поэтому distortion я вычислял по правилу, описанному в более новой версии документации. надеюсь, это не критично

import numpy as np 
from scipy.cluster.vq import kmeans #использую только для сравнения результатов

def my_kmeans(A, k, s = 20, eps = 1e-05): 
    N = A.shape[0] #столько точек
    m = A.shape[1] #такая размерность
    centroids = A[:k, :] #выбираю изначально центроиды как первые k точек 
    A = np.expand_dims(A, axis = 1) #добавляю размерность посерединке
    centroids1 = np.zeros((k, m)) #вспомогательная матрица для сдвига центроидов
    i = 0
    distortion = m * N * k * np.max(A) ** 2
    distortion1 = 0 #вспомогательный параметр для вычисления изменения distortion
    while distortion1 - distortion > eps or i < s:
        distortion1 = distortion
        centroids1 = centroids
        dif = A - centroids
        numcen = np.argmin((dif ** 2).sum(axis = 2), axis = 1) 
        C = np.zeros(N * m * k).reshape(N, k, m)
        C[np.arange(N), numcen, :] = dif[np.arange(N),  numcen, :] / np.bincount(numcen)[numcen].reshape(N, 1)
        centroids = centroids1 + np.squeeze(C.sum(axis = 0))        
        #найдены центроиды. ищем distortion 
        numcen = np.argmin(((A - centroids) ** 2).sum(axis = 2), axis = 1)
        C = np.zeros(N * m * k).reshape(N, k, m)
        C[np.arange(N), numcen, :] = (A - centroids)[np.arange(N),  numcen, :]
        distortion = np.sqrt((C ** 2).sum(axis = 2)).sum(axis = 1).mean() 
        
        i = i + 1
    return centroids, distortion
           
B = np.genfromtxt('dataset-for-hm_2.csv', delimiter=',', skip_header=1)
B = B / np.std(B, axis = 0)
                             
print(my_kmeans(B, 2)) 
print(kmeans(B, 2))

(array([[3.77111925, 5.90130403],
       [1.80133059, 4.02299172]]), 0.4704687987291571)
(array([[1.80133059, 4.02299172],
       [3.77111925, 5.90130403]]), 0.47046879872915703)
